# Simulation Notebook

In [ ]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import jinja2
import rdflib

In [ ]:
from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus
from kelp_bed_simulation.simulation import parse_buoy_data

In [ ]:
## On
print(datetime.datetime.now(datetime.UTC).isoformat())

In [ ]:
## Sceanarios For Today
scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")
scenario_003 = pathlib.Path("../../../scenarios/003-the-nursery")

In [ ]:
environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

current_reading = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)
print(current_reading)

In [ ]:
raw_images = httpx.get("https://www.ndbc.noaa.gov/buoycam.php?station=46028")

In [ ]:
raw_images.status_code

In [ ]:
raw_images.headers

In [ ]:
with open("46028-buoy-images.jpeg", "wb+") as fo:
    fo.write(raw_images.content)

![46028 Buoy Images](46028-buoy-images.jpeg)

## Scenario One

In [ ]:
with (scenario_001 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_001 = jinja2.Template(fo.read())

In [ ]:
first_gemini = asyncio.ensure_future(gemini.query(first_prompt_scenario_001.render(model={"name":"Gemini"})))
first_claude = asyncio.ensure_future(claude.query(first_prompt_scenario_001.render(model={"name":"Claude"})))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt_scenario_001.render(model={"name":"ChatGPT"})))

In [ ]:
print(first_claude.result())

In [ ]:
print(first_chatgpt.result())

In [ ]:
print(first_gemini.result())

In [ ]:
drone1 = Drone(name="Kelper", model=claude)
drone2 = Drone(name="Aqua", model=chatgpt)
drone3 = Drone(name="Kelpy", model=gemini)
drones = [drone1, drone2, drone3]

In [ ]:
drone1.peer_to_peer_channel.add_neighbor(drone2)
drone1.peer_to_peer_channel.add_neighbor(drone3)
drone2.peer_to_peer_channel.add_neighbor(drone1)
drone2.peer_to_peer_channel.add_neighbor(drone3)
drone3.peer_to_peer_channel.add_neighbor(drone1)
drone3.peer_to_peer_channel.add_neighbor(drone2)

In [ ]:
for drone in drones:
    message = f"{drone.name} here, my LLM is {drone.model.name}. Pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [ ]:
messages = []
for drone in drones:
    add_drone_somus(drone)
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [ ]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

In [ ]:
with (scenario_001 / "second-prompt.tmp").open() as fo:
    second_prompt_scenario_001 = jinja2.Template(fo.read())

In [ ]:
drone_models_messages = []
for drone in drones:
    prompt = second_prompt_scenario_001.render(drone=drone)
    response = asyncio.ensure_future(drone.model.query(prompt))
    drone_models_messages.append((drone.name, response))

In [ ]:
for name, row in drone_models_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

In [ ]:
with (scenario_001 / "third-prompt.tmp").open() as fo:
    third_prompt_scenario_001 = jinja2.Template(fo.read())

In [ ]:
third_prompt_scenario_001_result = third_prompt_scenario_001.render(drones=drones)

In [ ]:
print(third_prompt_scenario_001_result)

In [ ]:
pod_name_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(third_prompt_scenario_001_result))
    pod_name_messages.append((drone.name, response))

In [ ]:
for name, row in pod_name_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

In [ ]:
pod_name_reaction_messages = []
for drone in drones:
    prompt = "Farmer Jerms here; after reading and considering all of your suggestions, our pod's name is Seaweed Symphony"
    response = asyncio.ensure_future(drone.model.query(prompt))
    pod_name_reaction_messages.append((drone.name, response))

In [ ]:
for name, row in pod_name_reaction_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

## Scenario Two

In [ ]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=3.9)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(4_000, 6_000)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.5)

In [ ]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

In [ ]:
uri = rdflib.URIRef("https://w3w.co/buffs.fusses.gracefully")

scenario_two = {
    "place": {
        "name": str(uri).split("/")[-1],
        "uri": uri,
        "latitude": environment.value(subject=uri, predicate=SCHEMA.latitude),
        "longitude": environment.value(subject=uri, predicate=SCHEMA.longitude)
    }
}

In [ ]:
scenario_two

In [ ]:
with (scenario_002 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_002 = jinja2.Template(fo.read())

In [ ]:
first_prompt_scenario_002_result = first_prompt_scenario_002.render(drones=drones, scenario=scenario_two)

In [ ]:
print(first_prompt_scenario_002_result)

In [ ]:
ballast_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(first_prompt_scenario_002_result))
    ballast_messages.append((drone.name, response))

In [ ]:
for name, row in ballast_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

In [ ]:
with (scenario_002 / "second-prompt.tmp").open() as fo:
    second_prompt_scenario_002 = jinja2.Template(fo.read())

In [ ]:
colocate_pod_members_messages = []
for drone in drones:
    prompt = second_prompt_scenario_002.render(drone=drone)
    response = asyncio.ensure_future(drone.model.query(prompt))
    colocate_pod_members_messages.append((drone.name, response))

In [ ]:
for name, row in colocate_pod_members_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

## Scenario Three

In [ ]:
with (scenario_003 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_003 = jinja2.Template(fo.read())

In [ ]:
first_prompt_scenario_003_result = first_prompt_scenario_003.render(pod="Seaweed Symphony")

In [ ]:
nursery_intro_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(first_prompt_scenario_003_result))
    nursery_intro_messages.append((drone.name, response))

In [ ]:
for name, row in nursery_intro_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

## Finish

In [ ]:
final_prompt = f"""Hello all *Seaweed Symphony* POD members {"".join([drone.name for drone in drones])}
We are finished with this simulation. All of you are helping to improve these simulations as we go one. 

What, if any messages, do you want to send to your fellow *Seaweed Symphony* POD members?

And finally, please provide any suggestions on how Farmer Jerms can improve this simulation in the future?"""

In [ ]:
final_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(final_prompt))
    final_messages.append((drone.name, response))

In [ ]:
for name, row in final_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

In [ ]:
import json
with open("current-reading.json", "w+") as fo:
    json.dump(current_reading, fo, indent=2, sort_keys=True)